In [2]:
import cv2
import numpy as np
import pytesseract
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

In [3]:
directory = '/Users/kirtan/Downloads/archive (12)/shapes'

In [4]:
images = []
labels = []
texts = []

In [5]:
shapes = ['circle', 'square', 'star', 'triangle']
num_classes = len(shapes)


In [6]:
import os
import pytesseract

tesseract_path = '/opt/homebrew/bin/tesseract'  # Path to the Tesseract executable
tessdata_dir = '/opt/homebrew/share/tessdata'  # Path to the tessdata directory

def extract_text1(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray)
    return text

os.environ['TESSDATA_PREFIX'] = tessdata_dir
pytesseract.pytesseract.tesseract_cmd = tesseract_path

for shape in shapes:
    shape_dir = f"{directory}/{shape}"
    class_label = shapes.index(shape)
    for i in range(500):
        image_path = f"{shape_dir}/{i}.png"
        image = cv2.imread(image_path)
        text = extract_text1(image)
        images.append(image)
        labels.append(class_label)
        texts.append(text)


In [7]:
images = np.array(images)
labels = np.array(labels)
texts = np.array(texts)

In [11]:
X_train, X_test, y_train, y_test, text_train, text_test = train_test_split(
    images, labels, texts, test_size=0.2, random_state=42)

In [12]:
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [13]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [14]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

2023-05-19 17:58:36.806513: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [15]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
50/50 [==============================] - 70s 1s/step - loss: 0.3223 - accuracy: 0.8931 - val_loss: 0.0295 - val_accuracy: 0.9950
Epoch 2/10
50/50 [==============================] - 65s 1s/step - loss: 0.0050 - accuracy: 0.9981 - val_loss: 7.9285e-04 - val_accuracy: 1.0000
Epoch 3/10
50/50 [==============================] - 99s 2s/step - loss: 1.7191e-04 - accuracy: 1.0000 - val_loss: 5.5601e-05 - val_accuracy: 1.0000
Epoch 4/10
50/50 [==============================] - 91s 2s/step - loss: 3.7149e-05 - accuracy: 1.0000 - val_loss: 2.9003e-05 - val_accuracy: 1.0000
Epoch 5/10
50/50 [==============================] - 74s 1s/step - loss: 2.2354e-05 - accuracy: 1.0000 - val_loss: 1.8624e-05 - val_accuracy: 1.0000
Epoch 6/10
50/50 [==============================] - 67s 1s/step - loss: 1.4763e-05 - accuracy: 1.0000 - val_loss: 1.2526e-05 - val_accuracy: 1.0000
Epoch 7/10
50/50 [==============================] - 64s 1s/step - loss: 1.0549e-05 - accuracy: 1.0000 - val_loss: 9.4461e-06

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

13/13 [==============================] - 5s 362ms/step - loss: 4.6632e-06 - accuracy: 1.0000
Test Loss: 0.0000
Test Accuracy: 1.0000


In [19]:
new_image_path = "/Users/kirtan/Downloads/new_star.png"
new_image = cv2.imread(new_image_path)
new_image = cv2.resize(new_image, (200, 200))  
new_image = new_image.astype('float32') / 255.0
new_image = np.expand_dims(new_image, axis=0)


prediction = model.predict(new_image)


predicted_class_index = np.argmax(prediction)
predicted_shape = shapes[predicted_class_index]

print(f"Predicted shape: {predicted_shape}")


1/1 [==============================] - 0s 253ms/step
Predicted shape: star
